# El reto de la matriz de varianzas covarianzas

La aplicación práctica de la técnica de optimización propuesta plantea muchos problemas.Vermos cuales son los retos principales.
La optimización de la cartera requiere la inversión de una matriz de covarianza, pero en la práctica esta matriz de covarianza no suele ser de rango completo. Si esto no se maneja adecuadamente, puede dar lugar a resultados inestables e indeseables al realizar la operación de inversión.Esto es especialmente difícil cuando se optimizan activos muy correlacionados, y puede dar lugar a carteras inestables e inverosímiles.

## Estimación del error en las rentabilidades esperadas

El error que cometemos cuando utilizamos datos históricos para estimar la rentabilidad esparada es:
$$\epsilon_{\mu}=\frac{\sigma}{\sqrt{T}} \tag{1}$$

donde $\epsilon_{\mu}$ es el error en la estimación, $\sigma$ es la volatilidad del activo y $T$ es la amplitud temporal utilizada para estimar la esperanza.

Si disponemos de 20 años de datos, y estimamos la media de un activo con una volatilidad del 20%, el error de estimación de la media es de casi el 4,5%. 

In [3]:
def emu(sigma, T):
   emu=sigma/(T)**0.5
   return emu

In [5]:
sigma=0.2
T=20
emu(sigma,T)

0.044721359549995794

Que ocurre si trabajamos con datos mensuales.

In [7]:
sigma_m= 0.2/(12)**0.5
T_m=20*12
emu(sigma_m,T_m)


0.00372677996249965

Para alcanzar un error de estimación del 1% para este activo, necesitaríamos 400 años de datos. Aún más problemático, un intervalo de confianza del 90% de la estimación de la media estaría cerca del +/- 7%, un rango del 14%.La magnitud de este error de estimación y los requisitos implícitos relativos a la duración del periodo de medición son realmente sorprendentes. La magnitud de este error de estimación y los requisitos implícitos en cuanto a la duración del periodo de medición son realmente sorprendentes.